## Assignment 3 - TicTacToe (Due Feb 28)
Your assignment for this module is to create a working prototype for a TicTacToe. See below for a starting point.

## My approach

I want to create a program that plays TicTacToe against a player.

1- I will first create an object that stores the state of the board.
    - All X positions
    - All 0 positions
    - All Free spaces.

2 - Then I will make a function that checks for the least moves for the opposing player to win.

3 - Then I will make a function that checks for the least moves for the computer to win.

4 - Logic for which move has highest priority.

5 - Play a move.

6 - Check for win condition.


In [ ]:
from ipywidgets import GridspecLayout
from ipywidgets import Button, widgets

# The regular button object doesn't have a few properties that we need
# for example, we need square buttons and each button needs a row number and a column number
# we create a new "class". 
# i.e. new type of object that is similar to the default class but has the properties we need
class ttt_button(Button):        
    def __init__(self, row, col):
        self.row = row
        self.col = col        
        super().__init__(layout = widgets.Layout(width='30px', height='30px'),  )

def check_win():
    """Checks for game end condition.
    
    Returns:
    "winner" if there is a win_vector with three of the same mark.
    "tie" if all win_vectors have no available legal moves.
    "no_winner" otherwise.
    """
    
    global grid
    full_lines = 0

    for vector in win_vectors.values():
        if vector["X"] == 3 or vector["O"] == 3:
            for i in range(3): 
                for j in range(3):
                    grid[i,j].disabled = True
            return "winner"
        elif len(vector["empty"]) == 0:
            full_lines += 1
    
    if full_lines == 8:
        return "tie"
    
    return "no_winner"

def switch_player(current_player):
    """Switches the active player and returns that player as a string."""
    
    if current_player == 'X':
        return 'O'
    else:
        return 'X'
 
def update_win_vectors(row , col, player_key):
    """This function searches for all win vectors with the passes location and updates the vectors that contain it."""

    for vector in win_vectors.values():
        if ( row , col ) in vector["empty"]:
            vector["empty"].remove(( row , col ))
            vector[player_key] += 1

def computer_logic():
    """This algorithm finds a move to play and simulates a click on that cell, returns False if there's no legal."""

    # Block player's imminent win.
    for vector in win_vectors.values():
        if vector["X"] == 2 and len(vector["empty"]) == 1:
            row, col = vector["empty"][0]
            on_cell_click(grid[row, col])
            return True

    # Win if possible.
    for vector in win_vectors.values():
        if vector["O"] == 2 and len(vector["empty"]) == 1:
            row, col = vector["empty"][0]
            on_cell_click(grid[row, col])
            return True

    # Find somewhere with an empty space to extend.
    for vector in win_vectors.values():
        if vector["O"] == 1 and len(vector["empty"]) == 2:
            row, col = vector["empty"][0]
            on_cell_click(grid[row, col])
            return True

    #make first available move.
    for vector in win_vectors.values():
        if len(vector["empty"]) > 0:
            row, col = vector["empty"][0]
            on_cell_click(grid[row, col])
            return True
        
    return False


def on_cell_click(b):
    global top_output, bottom_output, grid, turn

    # disable the button so it can't be clicked again
    b.disabled = True

    # set the correct icon, set next_turn for future use
    if turn == 'X':        
        b.icon = 'times'
        update_win_vectors( b.row, b.col, 'X')
    else: 
        b.icon = 'circle-thin'
        update_win_vectors( b.row, b.col, 'O')

    #Check for win condition in win_vectors() and disable all buttons
    game_state = check_win()

    if game_state == "winner":
        bottom_output.clear_output()
        with bottom_output:
            print(f"The winner is {turn}'s!")
        return
    elif game_state == "tie":
        bottom_output.clear_output()
        with bottom_output:
            print(f"The game is a tie!")
        return

    # Changes the active player to the computer.
    turn = switch_player(turn)

    top_output.clear_output()
    with top_output:
        print("{} plays".format(turn))                
    
    bottom_output.clear_output()
    with bottom_output:
        print("Button {},{} clicked".format(b.row, b.col))

    # We only want to call the computer logic function if it's their turn... to avoid infinite recursion, and a very bland expirience for the player.
    if turn == "O":
        computer_logic()

# Main function in python
# Always at the end
if __name__ == "__main__":

    #create a 3x3 grid
    grid = GridspecLayout(3, 3, height='105px', width = '105px')

    # This dictionary turns all winning lines into a separate entry in the dictionary. It stores the number of X's, O's, and remaining legal spaces.
    global win_vectors
    win_vectors = {
        "diag_SE" : { "X": 0 , "O": 0 , "empty": [(0 , 0) , (1 , 1) , (2 , 2) ] },
        "diag_SW" : { "X": 0 , "O": 0 , "empty": [(0 , 2) , (1 , 1) , (2 , 0) ] },
        "row_0" : { "X": 0 , "O": 0 , "empty": [(0 , 0) , (0 , 1) , (0 , 2) ] },
        "row_1" : { "X": 0 , "O": 0 , "empty": [(1 , 0) , (1 , 1) , (1 , 2) ] },
        "row_2" : { "X": 0 , "O": 0 , "empty": [(2 , 0) , (2 , 1) , (2 , 2) ] },
        "col_0" : { "X": 0 , "O": 0 , "empty": [(0 , 0) , (1 , 0) , (2 , 0) ] },
        "col_1" : { "X": 0 , "O": 0 , "empty": [(0 , 1) , (1 , 1) , (2 , 1) ] },
        "col_2" : { "X": 0 , "O": 0 , "empty": [(0 , 2) , (1 , 2) , (2 , 2) ] }
    }

    #for each space in the grid
    for i in range(3): 
        for j in range(3):
            grid[i, j] = ttt_button(i,j)
            grid[i, j].on_click(on_cell_click)

    # two more UI elements. Feel free to use either/both
    top_output = widgets.Output()  
    bottom_output = widgets.Output() 

    turn = 'X'

    display(top_output)
    top_output.clear_output()
    with top_output:
        print("{} plays".format(turn))
        
    display(grid)
    display(bottom_output)

Output()

GridspecLayout(children=(ttt_button(layout=Layout(grid_area='widget001', height='30px', width='30px'), style=B…

Output()